In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.optimize as optimization
import snowflake.connector
import yfinance as yf  # Faltaba este import
import os
import getpass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuración global
NUM_TRADING_DAYS = 252
NUM_PORTFOLIOS = 10000

# Fechas para el análisis
start_date = '2020-01-01'
end_date = '2024-12-31'

# Tickers del IBEX 35 más líquidos
stocks = ['SAN.MC', 'BBVA.MC', 'IBE.MC', 'ITX.MC', 'TEF.MC', 'REP.MC']

print(f"📊 Configuración:")
print(f"   • Período: {start_date} a {end_date}")
print(f"   • Acciones: {stocks}")
print(f"   • Portafolios a simular: {NUM_PORTFOLIOS:,}")
print(f"   • Días de trading anuales: {NUM_TRADING_DAYS}")

In [38]:
password = getpass.getpass("🔒 Introduce tu contraseña de Snowflake: ")

conn = snowflake.connector.connect(
    user='TFMGRUPO4',
    password=password,
    account='WYNIFVB-YE01854',
    warehouse='COMPUTE_WH',
    database='YAHOO_FINANCE',
    schema='MACHINE_LEARNING',
    role='ACCOUNTADMIN'
)

# Ejecutar consulta simple para ver columnas
query = """
    SELECT * 
    FROM TICKERS_INDEX
    LIMIT 5
"""
cursor = conn.cursor()
cursor.execute(query)

# Crear DataFrame
raw_df = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
cursor.close()
conn.close()


raw_df.head()


,TICKER,CLOSE,HIGH,LOW,OPEN,VOLUME,FECHA
0,1COV.DE,42.180000310,42.610000610,41.500000000,41.500000000,1238476,2020-01-02
1,1YD.DE,28.475000380,29.200000760,27.665000920,28.500000000,380,2020-01-02
2,33L.DE,58.299999240,58.299999240,58.299999240,58.299999240,0,2020-01-02
3,4I1.DE,76.319999690,77.419998170,76.319999690,77.239997860,628,2020-01-02
4,7HP.DE,18.555999760,18.601999280,18.215999600,18.500000000,1681,2020-01-02


In [41]:
def download_data():
    # nombre de la acción (clave) - valores de la acción (2019-2024) como valores
    stock_data = {}

    for stock in stocks:
        # precios de cierre
        ticker = yf.Ticker(stock)
        stock_data[stock] = ticker.history(start=start_date, end=end_date)['Close']

    return pd.DataFrame(stock_data)


def show_data(data):
    data.plot(figsize=(10, 5))
    plt.show()


def calculate_return(data):
    # NORMALIZACIÓN - para medir todas las variables en métrica comparable
    log_return = np.log(data / data.shift(1))
    return log_return[1:]


def show_statistics(returns):
    # en lugar de métricas diarias buscamos métricas anuales
    # media del retorno anual
    print(returns.mean() * NUM_TRADING_DAYS)
    print(returns.cov() * NUM_TRADING_DAYS)


def show_mean_variance(returns, weights):
    # buscamos el retorno anual
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov()
                                                            * NUM_TRADING_DAYS, weights)))
    print("Retorno esperado del portafolio (rentabilidad): ", portfolio_return)
    print("Volatilidad esperada del portafolio (desviación estándar): ", portfolio_volatility)


def show_portfolios(returns, volatilities):
    plt.figure(figsize=(10, 6))
    plt.scatter(volatilities, returns, c=returns / volatilities, marker='o')
    plt.grid(True)
    plt.xlabel('Volatilidad Esperada')
    plt.ylabel('Retorno Esperado')
    plt.colorbar(label='Ratio de Sharpe')
    plt.show()


def generate_portfolios(returns):
    portfolio_means = []
    portfolio_risks = []
    portfolio_weights = []

    for _ in range(NUM_PORTFOLIOS):
        w = np.random.random(len(stocks))
        w /= np.sum(w)
        portfolio_weights.append(w)
        portfolio_means.append(np.sum(returns.mean() * w) * NUM_TRADING_DAYS)
        portfolio_risks.append(np.sqrt(np.dot(w.T, np.dot(returns.cov()
                                                          * NUM_TRADING_DAYS, w))))

    return np.array(portfolio_weights), np.array(portfolio_means), np.array(portfolio_risks)


def statistics(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * NUM_TRADING_DAYS
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov()
                                                            * NUM_TRADING_DAYS, weights)))
    return np.array([portfolio_return, portfolio_volatility,
                     portfolio_return / portfolio_volatility])


# el módulo scipy optimize puede encontrar el mínimo de una función dada
# el máximo de f(x) es el mínimo de -f(x)
def min_function_sharpe(weights, returns):
    return -statistics(weights, returns)[2]


# ¿cuáles son las restricciones? ¡La suma de pesos = 1!
# f(x)=0 esta es la función a minimizar
def optimize_portfolio(weights, returns):
    # la suma de pesos es 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # los pesos pueden ser 1 como máximo: 1 cuando el 100% del dinero se invierte en una sola acción
    bounds = tuple((0, 1) for _ in range(len(stocks)))
    return optimization.minimize(fun=min_function_sharpe, x0=weights[0], args=returns
                                 , method='SLSQP', bounds=bounds, constraints=constraints)


def print_optimal_portfolio(optimum, returns):
    print("Portafolio óptimo: ", optimum['x'].round(3))
    print("Retorno esperado, volatilidad y ratio de Sharpe: ",
          statistics(optimum['x'].round(3), returns))


def show_optimal_portfolio(opt, rets, portfolio_rets, portfolio_vols):
    plt.figure(figsize=(10, 6))
    plt.scatter(portfolio_vols, portfolio_rets, c=portfolio_rets / portfolio_vols, marker='o')
    plt.grid(True)
    plt.xlabel('Volatilidad Esperada')
    plt.ylabel('Retorno Esperado')
    plt.colorbar(label='Ratio de Sharpe')
    plt.plot(statistics(opt['x'], rets)[1], statistics(opt['x'], rets)[0], 'g*', markersize=20.0)
    plt.show()


if __name__ == '__main__':
    dataset = download_data()
    show_data(dataset)
    log_daily_returns = calculate_return(dataset)
    # show_statistics(log_daily_returns)

    pweights, means, risks = generate_portfolios(log_daily_returns)
    show_portfolios(means, risks)
    optimum = optimize_portfolio(pweights, log_daily_returns)
    print_optimal_portfolio(optimum, log_daily_returns)
    show_optimal_portfolio(optimum, log_daily_returns, means, risks)


NameError: name 'stocks' is not defined